In [ ]:
# データセット作成

!pip install scipy

clip_size = 40960 # clips.npy ファイルを DL してきた場合はこの値は変更しないでください
window_size = 4096
window_num_per_clip = int(clip_size / window_size)

from scipy.io.wavfile import read
import glob
import os
import numpy as np

if os.path.isfile("../data/arrays/clips.npy"):
    print("clips.npy exists.")
    clips = np.load("../data/arrays/clips.npy")
else:
    files = glob.glob("../data/wav44100/*")
    #files = ["../data/wav44100/3DEmbodimentFromLines.wav"]
    raw_data_list = [read(file)[1] for file in files]

    clips = np.zeros((0, clip_size), dtype=np.int16)
    for raw_data in raw_data_list:
        tmp = [raw_data[i:i + clip_size] for i in range(0, len(raw_data) - clip_size, clip_size)]
        clips = np.vstack((clips, np.array(tmp)))
    np.save("../data/arrays/clips", clips)
    
if os.path.isfile("../data/arrays/fft_log_" + str(window_size) + ".npy"):
    print("fft_log_" + str(window_size) + ".npy exists.")
    fft = np.load("../data/arrays/fft_log_" + str(window_size) + ".npy")
else:
    fft = np.zeros((0, window_num_per_clip, window_size))
    num = 0
    for clip in clips:
        print("fft progress: clip " + str(num))
        
        fft_tmp = np.zeros((0, window_size))
        tmp = [clip[i:i + window_size] for i in range(0, len(clip) - window_size + 1, window_size)]
        for tmp2 in tmp:
            spectrum = np.abs(np.fft.fft(tmp2))
            spectrum = np.log(spectrum + 1) / np.log(np.finfo(np.float64).max)
            fft_tmp = np.vstack((fft_tmp, spectrum))
        fft = np.vstack((fft, fft_tmp.reshape(1, window_num_per_clip, window_size)))
        num += 1

    np.save("../data/arrays/fft_log_" + str(window_size), fft)
    
print("clips.shape: " + str(clips.shape))
print("fft.shape: " + str(fft.shape))

     |████████████████████████████████| 34.5 MB 16.1 MB/s eta 0:00:01    |██████                          | 6.4 MB 11.4 MB/s eta 0:00:03
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
clips.npy exists.
fft progress: clip 0
fft progress: clip 1
fft progress: clip 2
fft progress: clip 3
fft progress: clip 4
fft progress: clip 5
fft progress: clip 6
fft progress: clip 7
fft progress: clip 8
fft progress: clip 9
fft progress: clip 10
fft progress: clip 11
fft progress: clip 12
fft progress: clip 13
fft progress: clip 14
fft progress: clip 15
fft progress: clip 16
fft progress: clip 17
fft progress: clip 18
fft progress: clip 19
fft progress: clip 20
fft progress: clip 21
fft progress: clip 22
fft progress: clip 23
fft progress: clip 24
fft progress: clip 25
fft progress: clip 26
fft progress: clip 27
fft progress: clip 28
fft progress: clip 29
fft progress: clip 30
fft progress: clip 31
fft progress: clip 32
fft progress: clip 33
fft progress

fft progress: clip 342
fft progress: clip 343
fft progress: clip 344
fft progress: clip 345
fft progress: clip 346
fft progress: clip 347
fft progress: clip 348
fft progress: clip 349
fft progress: clip 350
fft progress: clip 351
fft progress: clip 352
fft progress: clip 353
fft progress: clip 354
fft progress: clip 355
fft progress: clip 356
fft progress: clip 357
fft progress: clip 358
fft progress: clip 359
fft progress: clip 360
fft progress: clip 361
fft progress: clip 362
fft progress: clip 363
fft progress: clip 364
fft progress: clip 365
fft progress: clip 366
fft progress: clip 367
fft progress: clip 368
fft progress: clip 369
fft progress: clip 370
fft progress: clip 371
fft progress: clip 372
fft progress: clip 373
fft progress: clip 374
fft progress: clip 375
fft progress: clip 376
fft progress: clip 377
fft progress: clip 378
fft progress: clip 379
fft progress: clip 380
fft progress: clip 381
fft progress: clip 382
fft progress: clip 383
fft progress: clip 384
fft progres

fft progress: clip 704
fft progress: clip 705
fft progress: clip 706
fft progress: clip 707
fft progress: clip 708
fft progress: clip 709
fft progress: clip 710
fft progress: clip 711
fft progress: clip 712
fft progress: clip 713
fft progress: clip 714
fft progress: clip 715
fft progress: clip 716
fft progress: clip 717
fft progress: clip 718
fft progress: clip 719
fft progress: clip 720
fft progress: clip 721
fft progress: clip 722
fft progress: clip 723
fft progress: clip 724
fft progress: clip 725
fft progress: clip 726
fft progress: clip 727
fft progress: clip 728
fft progress: clip 729
fft progress: clip 730
fft progress: clip 731
fft progress: clip 732
fft progress: clip 733
fft progress: clip 734
fft progress: clip 735
fft progress: clip 736
fft progress: clip 737
fft progress: clip 738
fft progress: clip 739
fft progress: clip 740
fft progress: clip 741
fft progress: clip 742
fft progress: clip 743
fft progress: clip 744
fft progress: clip 745
fft progress: clip 746
fft progres

fft progress: clip 1061
fft progress: clip 1062
fft progress: clip 1063
fft progress: clip 1064
fft progress: clip 1065
fft progress: clip 1066
fft progress: clip 1067
fft progress: clip 1068
fft progress: clip 1069
fft progress: clip 1070
fft progress: clip 1071
fft progress: clip 1072
fft progress: clip 1073
fft progress: clip 1074
fft progress: clip 1075
fft progress: clip 1076
fft progress: clip 1077
fft progress: clip 1078
fft progress: clip 1079
fft progress: clip 1080
fft progress: clip 1081
fft progress: clip 1082
fft progress: clip 1083
fft progress: clip 1084
fft progress: clip 1085
fft progress: clip 1086
fft progress: clip 1087
fft progress: clip 1088
fft progress: clip 1089
fft progress: clip 1090
fft progress: clip 1091
fft progress: clip 1092
fft progress: clip 1093
fft progress: clip 1094
fft progress: clip 1095
fft progress: clip 1096
fft progress: clip 1097
fft progress: clip 1098
fft progress: clip 1099
fft progress: clip 1100
fft progress: clip 1101
fft progress: cl

fft progress: clip 1405
fft progress: clip 1406
fft progress: clip 1407
fft progress: clip 1408
fft progress: clip 1409
fft progress: clip 1410
fft progress: clip 1411
fft progress: clip 1412
fft progress: clip 1413
fft progress: clip 1414
fft progress: clip 1415
fft progress: clip 1416
fft progress: clip 1417
fft progress: clip 1418
fft progress: clip 1419
fft progress: clip 1420
fft progress: clip 1421
fft progress: clip 1422
fft progress: clip 1423
fft progress: clip 1424
fft progress: clip 1425
fft progress: clip 1426
fft progress: clip 1427
fft progress: clip 1428
fft progress: clip 1429
fft progress: clip 1430
fft progress: clip 1431
fft progress: clip 1432
fft progress: clip 1433
fft progress: clip 1434
fft progress: clip 1435
fft progress: clip 1436
fft progress: clip 1437
fft progress: clip 1438
fft progress: clip 1439
fft progress: clip 1440
fft progress: clip 1441
fft progress: clip 1442
fft progress: clip 1443
fft progress: clip 1444
fft progress: clip 1445
fft progress: cl

...

fft progress: clip 3460
fft progress: clip 3461
fft progress: clip 3462
fft progress: clip 3463
fft progress: clip 3464
fft progress: clip 3465
fft progress: clip 3466
fft progress: clip 3467
fft progress: clip 3468
fft progress: clip 3469
fft progress: clip 3470
fft progress: clip 3471
fft progress: clip 3472
fft progress: clip 3473
fft progress: clip 3474
fft progress: clip 3475
fft progress: clip 3476
fft progress: clip 3477
fft progress: clip 3478
fft progress: clip 3479
fft progress: clip 3480
fft progress: clip 3481
fft progress: clip 3482
fft progress: clip 3483
fft progress: clip 3484
fft progress: clip 3485
fft progress: clip 3486
fft progress: clip 3487
fft progress: clip 3488
fft progress: clip 3489
fft progress: clip 3490
fft progress: clip 3491
fft progress: clip 3492
fft progress: clip 3493
fft progress: clip 3494
fft progress: clip 3495
fft progress: clip 3496
fft progress: clip 3497
fft progress: clip 3498
fft progress: clip 3499
fft progress: clip 3500
fft progress: cl

fft progress: clip 3803
fft progress: clip 3804
fft progress: clip 3805
fft progress: clip 3806
fft progress: clip 3807
fft progress: clip 3808
fft progress: clip 3809
fft progress: clip 3810
fft progress: clip 3811
fft progress: clip 3812
fft progress: clip 3813
fft progress: clip 3814
fft progress: clip 3815
fft progress: clip 3816
fft progress: clip 3817
fft progress: clip 3818
fft progress: clip 3819
fft progress: clip 3820
fft progress: clip 3821
fft progress: clip 3822
fft progress: clip 3823
fft progress: clip 3824
fft progress: clip 3825
fft progress: clip 3826
fft progress: clip 3827
fft progress: clip 3828
fft progress: clip 3829
fft progress: clip 3830
fft progress: clip 3831
fft progress: clip 3832
fft progress: clip 3833
fft progress: clip 3834
fft progress: clip 3835
fft progress: clip 3836
fft progress: clip 3837
fft progress: clip 3838
fft progress: clip 3839
fft progress: clip 3840
fft progress: clip 3841
fft progress: clip 3842
fft progress: clip 3843
fft progress: cl

fft progress: clip 4146
fft progress: clip 4147
fft progress: clip 4148
fft progress: clip 4149
fft progress: clip 4150
fft progress: clip 4151
fft progress: clip 4152
fft progress: clip 4153
fft progress: clip 4154
fft progress: clip 4155
fft progress: clip 4156
fft progress: clip 4157
fft progress: clip 4158
fft progress: clip 4159
fft progress: clip 4160
fft progress: clip 4161
fft progress: clip 4162
fft progress: clip 4163
fft progress: clip 4164
fft progress: clip 4165
fft progress: clip 4166
fft progress: clip 4167
fft progress: clip 4168
fft progress: clip 4169
fft progress: clip 4170
fft progress: clip 4171
fft progress: clip 4172
fft progress: clip 4173
fft progress: clip 4174
fft progress: clip 4175
fft progress: clip 4176
fft progress: clip 4177
fft progress: clip 4178
fft progress: clip 4179
fft progress: clip 4180
fft progress: clip 4181
fft progress: clip 4182
fft progress: clip 4183
fft progress: clip 4184
fft progress: clip 4185
fft progress: clip 4186
fft progress: cl

fft progress: clip 4489
fft progress: clip 4490
fft progress: clip 4491
fft progress: clip 4492
fft progress: clip 4493
fft progress: clip 4494
fft progress: clip 4495
fft progress: clip 4496
fft progress: clip 4497
fft progress: clip 4498
fft progress: clip 4499
fft progress: clip 4500
fft progress: clip 4501
fft progress: clip 4502
fft progress: clip 4503
fft progress: clip 4504
fft progress: clip 4505
fft progress: clip 4506
fft progress: clip 4507
fft progress: clip 4508
fft progress: clip 4509
fft progress: clip 4510
fft progress: clip 4511
fft progress: clip 4512
fft progress: clip 4513
fft progress: clip 4514
fft progress: clip 4515
fft progress: clip 4516
fft progress: clip 4517
fft progress: clip 4518
fft progress: clip 4519
fft progress: clip 4520
fft progress: clip 4521
fft progress: clip 4522
fft progress: clip 4523
fft progress: clip 4524
fft progress: clip 4525
fft progress: clip 4526
fft progress: clip 4527
fft progress: clip 4528
fft progress: clip 4529
fft progress: cl

fft progress: clip 4831
fft progress: clip 4832
fft progress: clip 4833
fft progress: clip 4834
fft progress: clip 4835
fft progress: clip 4836
fft progress: clip 4837
fft progress: clip 4838
fft progress: clip 4839
fft progress: clip 4840
fft progress: clip 4841
fft progress: clip 4842
fft progress: clip 4843
fft progress: clip 4844
fft progress: clip 4845
fft progress: clip 4846
fft progress: clip 4847
fft progress: clip 4848
fft progress: clip 4849
fft progress: clip 4850
fft progress: clip 4851
fft progress: clip 4852
fft progress: clip 4853
fft progress: clip 4854
fft progress: clip 4855
fft progress: clip 4856
fft progress: clip 4857
fft progress: clip 4858
fft progress: clip 4859
fft progress: clip 4860
fft progress: clip 4861
fft progress: clip 4862
fft progress: clip 4863
fft progress: clip 4864
fft progress: clip 4865
fft progress: clip 4866
fft progress: clip 4867
fft progress: clip 4868
fft progress: clip 4869
fft progress: clip 4870
fft progress: clip 4871
fft progress: cl

In [ ]:
# 学習データとテストデータに分ける

train_data_rate = 0.8

fft_sum_threshold = 100 # 1 window の sum(log(fft)) が閾値に満たないときに除外します

def normalize(array):
    if (np.sum(array) == 0):
        return array
    else:
        return array / np.sum(array)

fft = np.concatenate([fft[:-1, :, :], fft[1:, 0, :].reshape(fft.shape[0] - 1, 1, fft.shape[2])], 1)

clips = np.delete(clips, np.where(np.sum(fft[:,0,:], axis=1) < fft_sum_threshold)[0], axis=0)
fft = np.delete(fft, np.where(np.sum(fft[:,0,:], axis=1) < fft_sum_threshold)[0], axis=0)

fft = np.apply_along_axis(normalize, 2, fft)

p = np.random.permutation(len(fft))
fft = fft[p]
clips = clips[p]

x_train = fft[:int(fft.shape[0] * train_data_rate), :window_num_per_clip, :].reshape(int(fft.shape[0] * train_data_rate), window_num_per_clip, window_size, 1)
x_test = fft[int(fft.shape[0] * train_data_rate):, :window_num_per_clip, :].reshape(fft.shape[0] - int(fft.shape[0] * train_data_rate), window_num_per_clip, window_size, 1)
y_train = fft[:int(fft.shape[0] * train_data_rate), window_num_per_clip, :]
y_test = fft[int(fft.shape[0] * train_data_rate):, window_num_per_clip, :]

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

print(np.max(x_train[0]))

In [ ]:
def plot_history(history):
    # 精度の履歴をプロット
    plt.plot(history.history['accuracy'],"o-",label="accuracy")
    plt.plot(history.history['val_accuracy'],"o-",label="val_acc")
    plt.title('model accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.legend(loc="lower right")
    plt.show()
 
    # 損失の履歴をプロット
    plt.plot(history.history['loss'],"o-",label="loss",)
    plt.plot(history.history['val_loss'],"o-",label="val_loss")
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(loc='lower right')
    plt.show()

In [ ]:
# モデル作成

import keras

from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, Input
from keras.optimizers import Adam

learning_rate = 0.0001
 
model = keras.models.Sequential()
model.add(Conv2D(3, (3, 1), activation='relu', input_shape=(window_num_per_clip, window_size, 1)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(units=512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(units=window_size))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer=Adam(learning_rate=learning_rate),metrics=['accuracy'])

model.build()
model.summary()

In [ ]:
# 学習
batch_size = 256
epochs = 100
model_name = "fft_batch" + str(batch_size) + "_e" + str(epochs)

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

model.save("../data/models/" + model_name)

In [ ]:
# loss 遷移のプロット

import matplotlib.pyplot as plt

plot_history(history)

In [ ]:
# 楽曲の出力

!pip install scikit-learn

import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from scipy.io import wavfile
from keras.models import load_model

model = load_model("../data/models/" + model_name)
file_name = "out_" + model_name

first_index = np.random.randint(0, len(x_test))
predict_index = first_index
out = np.zeros((0, ), dtype=np.int16)

for i in range(10):
    predict = model.predict(np.array([x_test[predict_index]]))
    
    index = 0
    similar_index = 0
    cos_sim = np.inf
    for spectrum in x_test[:, 0:1, :, 0]:
        tmp_sim = cosine_similarity(predict, spectrum) 
        if cos_sim > tmp_sim:
            cos_sim = tmp_sim
            similar_index = index
        index += 1
    
    print(similar_index)
    
    predict_index = similar_index
    out = np.hstack((out, clips[predict_index + int(fft.shape[0] * train_data_rate)]))
    
wavfile.write("../data/out/" + file_name + "_1.wav", 44100, out)

first_index = np.random.randint(0, len(x_test))
predict_index = first_index
out = np.zeros((0, ), dtype=np.int16)

for i in range(10):
    predict = model.predict(np.array([x_test[predict_index]]))
    
    index = 0
    similar_index = 0
    cos_sim = np.inf
    for spectrum in x_test[:, 0:1, :, 0]:
        tmp_sim = cosine_similarity(predict, spectrum) 
        if cos_sim > tmp_sim:
            cos_sim = tmp_sim
            similar_index = index
        index += 1
    
    print(similar_index)
    
    predict_index = similar_index
    out = np.hstack((out, clips[predict_index + int(fft.shape[0] * train_data_rate)]))
    
wavfile.write("../data/out/" + file_name + "_2.wav", 44100, out)

first_index = np.random.randint(0, len(x_test))
predict_index = first_index
out = np.zeros((0, ), dtype=np.int16)

for i in range(10):
    predict = model.predict(np.array([x_test[predict_index]]))
    
    index = 0
    similar_index = 0
    cos_sim = np.inf
    for spectrum in x_test[:, 0:1, :, 0]:
        tmp_sim = cosine_similarity(predict, spectrum) 
        if cos_sim > tmp_sim:
            cos_sim = tmp_sim
            similar_index = index
        index += 1
    
    print(similar_index)
    
    predict_index = similar_index
    out = np.hstack((out, clips[predict_index + int(fft.shape[0] * train_data_rate)]))
    
wavfile.write("../data/out/" + file_name + "_3.wav", 44100, out)